# Interactive Life Expectancy Simulator

Adjust health predictors to see how the predicted life expectancy changes and view the distribution of the predictor you just changed.

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, HTML

# Load data
coefs_df = pd.read_csv('health_unscaled_coefficients.csv')
data_df  = pd.read_csv('health_predictors.csv')
sns.set_theme()

In [7]:
# Extract intercept
intercept = coefs_df.loc[coefs_df['variable']=='Intercept', 'beta_unscaled'].iat[0]
# Build slopes dict for all other variables
slopes = (
    coefs_df[coefs_df['variable']!='Intercept']
    .set_index('variable')['beta_unscaled']
    .to_dict()
)
# Exclude unwanted variable if present
slopes.pop('Lifetime risk of maternal death (1 in: rate varies by country)', None)
predictors = list(slopes.keys())
predictors

['Population ages 0-14, total',
 'People practicing open defecation (% of population)',
 'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
 'Domestic private health expenditure per capita (current US$)',
 'Prevalence of overweight (modeled estimate, % of children under 5)',
 'Number of maternal deaths',
 'Lifetime risk of maternal death (%)',
 'Incidence of tuberculosis (per 100,000 people)',
 'Prevalence of undernourishment (% of population)']

In [8]:
def predict_le(vals):
    '''Compute predicted life expectancy given a dict of predictor values.'''
    return intercept + sum(slopes[var] * vals[var] for var in predictors)

In [9]:
sliders = {}
for var in predictors:
    col = data_df[var].dropna()
    min_val, max_val = float(col.min()), float(col.max())
    mu = coefs_df.loc[coefs_df['variable']==var, 'mean'].iat[0]
    step = (max_val - min_val) / 100.0
    sliders[var] = widgets.FloatSlider(
        value=mu,
        min=min_val,
        max=max_val,
        step=step,
        description=var,
        continuous_update=False,
        layout=widgets.Layout(width='1200px'),
        style={'description_width': '600px'}
    )
ui = widgets.VBox(list(sliders.values()))
display(ui)

In [10]:
out = widgets.Output()
display(out)

def update(var):
    out.clear_output()
    # gather current slider values
    vals = {v: sliders[v].value for v in predictors}
    pred = predict_le(vals)
    with out:
        display(HTML(f"<h2 style='color:darkblue;'>Predicted Life Expectancy: {pred:.2f} years</h2>"))
        plt.figure(figsize=(6,4))
        sns.histplot(data_df[var].dropna(), bins=30, kde=True)
        plt.axvline(sliders[var].value, color='red', linestyle='--')
        plt.title(f"Distribution of {var}")
        plt.xlabel(var)
        plt.tight_layout()
        plt.show()

# detach any old observers, then re‐attach exactly one per slider
for w in sliders.values():
    w.unobserve_all()

def make_handler(var):
    def handler(change):
        update(var)
    return handler

for var, w in sliders.items():
    w.observe(make_handler(var), names='value')

# initial draw
update(predictors[0])


Output()